In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import cv2

# Grid dimensions
GRID_HEIGHT = 6
GRID_WIDTH = 6

# Cell types
EMPTY = 0
OBSTACLE = 1
PICKUP = 2
DROPOFF = 3
ROBOT = 4

# Actions
ACTIONS = ['up', 'down', 'left', 'right']
ACTION_IDX = {a: i for i, a in enumerate(ACTIONS)}

# Load Q-table
q_table = np.load("q_table.npy")

# Create folder to save test frames
test_frame_dir = "testing_frames"
os.makedirs(test_frame_dir, exist_ok=True)

# Environment setup
def create_grid():
    grid = np.zeros((GRID_HEIGHT, GRID_WIDTH), dtype=int)
    grid[1, 1] = OBSTACLE
    grid[2, 3] = OBSTACLE
    grid[3, 1] = OBSTACLE
    grid[1, 4] = OBSTACLE
    grid[0, 5] = PICKUP
    grid[5, 0] = DROPOFF
    return grid

def is_valid(y, x):
    return 0 <= y < GRID_HEIGHT and 0 <= x < GRID_WIDTH

def move(y, x, action):
    if action == 'up': y -= 1
    elif action == 'down': y += 1
    elif action == 'left': x -= 1
    elif action == 'right': x += 1
    return y, x

# Visualization
cmap = mcolors.ListedColormap(['white', 'black', 'green', 'red', 'blue'])

def render(grid, path, final_pos, carried_once, step):
    vis_grid = grid.copy()
    y, x = final_pos
    vis_grid[y, x] = ROBOT

    plt.imshow(vis_grid, cmap=cmap, vmin=0, vmax=4)

    # Mark start point
    start_y, start_x = path[0]
    plt.plot(start_x, start_y, marker='o', color='purple', markersize=8, label='Start')

    for i in range(len(path) - 1):
        y1, x1 = path[i]
        y2, x2 = path[i + 1]
        dx = x2 - x1
        dy = y2 - y1
        plt.arrow(x1, y1, dx * 0.3, dy * 0.3, head_width=0.15, head_length=0.15, fc='black', ec='black')

    plt.title(f"Test Step {step} | Final: ({y},{x}) | Carrying: {'Yes' if carried_once else 'No'}")
    frame_path = os.path.join(test_frame_dir, f"test_frame_{step:03d}.png")
    plt.savefig(frame_path)
    plt.clf()

# Testing loop
grid = create_grid()
robot_y, robot_x = 0, 0
carrying = 0
carried_once = False
path = [(robot_y, robot_x)]

for step in range(100):
    state = (robot_y, robot_x, carrying)
    action_idx = np.argmax(q_table[robot_y, robot_x, carrying])
    action = ACTIONS[action_idx]

    new_y, new_x = move(robot_y, robot_x, action)
    if not is_valid(new_y, new_x) or grid[new_y, new_x] == OBSTACLE:
        new_y, new_x = robot_y, robot_x

    if grid[new_y, new_x] == PICKUP and carrying == 0:
        carrying = 1
        carried_once = True
    elif grid[new_y, new_x] == DROPOFF and carrying == 1:
        carrying = 0
        break

    
    robot_y, robot_x = new_y, new_x
    path.append((robot_y, robot_x))
    render(grid, path, (robot_y, robot_x), carried_once, step)

plt.close()

print("Testing completed.");
# Compile test images into video
test_frame_files = sorted([os.path.join(test_frame_dir, f) for f in os.listdir(test_frame_dir) if f.endswith('.png')])
if test_frame_files:
    height, width, _ = cv2.imread(test_frame_files[0]).shape
    out = cv2.VideoWriter('testing_video.avi', cv2.VideoWriter_fourcc(*'mp4v'), 2, (width, height))
    for filename in test_frame_files:
        out.write(cv2.imread(filename))
    out.release()

print("Video saved as 'test_result.avi'")


Testing completed.
Video saved as 'test_result.avi'
